In [1]:
import numpy as np

# --- model (constant intensities) ---
sigma = 0.2
mu = 0.0
a = mu - 0.5*sigma**2          # drift in y-PDE
lam1_hat, lam2_hat = 0.18, 0.12
eta = np.array([8.0, 6.0])
delta = np.array([0.02, 0.03])

# --- grid in y ---
y_max, M = 0.8, 600
y = np.linspace(0.0, y_max, M+1)
Dy = y[1] - y[0]

m_delta = np.ceil(delta / Dy).astype(int)
Eshoot = [np.exp(-eta[j] * np.maximum(y - delta[j], 0.0)) for j in (0,1)]

def solve_tridiag(sub, diag, sup, rhs):
    sub = sub.copy(); diag = diag.copy(); sup = sup.copy(); d = rhs.copy()
    for m in range(1, M+1):
        w = sub[m] / diag[m-1]
        diag[m] -= w * sup[m-1]
        d[m]    -= w * d[m-1]
    x = np.empty_like(d)
    x[M] = d[M] / diag[M]
    for m in range(M-1, -1, -1):
        x[m] = (d[m] - sup[m] * x[m+1]) / diag[m]
    return x

def build_y_LHS(dt, sink_diag, dtype):
    c2 = 0.5 * sigma**2
    sub = (-dt * (c2 / Dy**2 - a / Dy)) * np.ones(M+1, dtype=dtype)
    sup = (-dt * (c2 / Dy**2)) * np.ones(M+1, dtype=dtype)
    diag = (1.0 + dt*(2*c2 / Dy**2)) * np.ones(M+1, dtype=dtype)
    diag += sink_diag
    # Dirichlet BCs
    diag[0] = 1.0; sub[0] = 0.0; sup[0] = 0.0
    diag[-1] = 1.0; sub[-1] = 0.0; sup[-1] = 0.0
    return sub, diag, sup

def update_g_implicit_y(f, j):
    """Backward-Euler in y: g'(y)=eta*(f(y-δ)_+-g). Works for complex f."""
    gj = np.zeros_like(f)
    md = m_delta[j]
    ej = eta[j]
    c = ej*Dy
    if md == 0:
        gj[0] = (c * f[0]) / (1.0 + c)
        for m in range(1, M+1):
            gj[m] = (gj[m-1] + c * f[m]) / (1.0 + c)
    else:
        gj[md] = (c * f[0]) / (1.0 + c)
        for m in range(md+1, M+1):
            gj[m] = (gj[m-1] + c * f[m - md]) / (1.0 + c)
    return gj

def solve_LT_at_q(qc, max_outer=200, tol=1e-11, inner_max=20, inner_tol=1e-12):
    """
    Solve for f(y; q=qc) with backward-implicit scheme.
    qc can be complex (Talbot needs this). Returns complex array f(y).
    """
    dtype = np.complex128 if np.iscomplexobj(qc) or getattr(qc, 'imag', 0.0) != 0.0 else np.float64
    lam_sum = lam1_hat + lam2_hat
    # choose a conservative dt based on sink scale
    dt = min(0.9/(abs(qc)+lam_sum), 0.5)
    sink_diag = dt * (qc + lam_sum) * np.ones(M+1, dtype=dtype)

    sub, diag, sup = build_y_LHS(dt, sink_diag, dtype=dtype)

    f = np.zeros(M+1, dtype=dtype); f[0] = 1.0; f[-1] = 0.0
    E1 = np.asarray(Eshoot[0], dtype=dtype)
    E2 = np.asarray(Eshoot[1], dtype=dtype)

    for it in range(max_outer):
        f_old = f.copy()
        # inner Picard for g^{n+1}(f^{n+1})
        for inner in range(inner_max):
            g1 = update_g_implicit_y(f, 0)
            g2 = update_g_implicit_y(f, 1)
            rhs = f_old + dt*( lam1_hat*(g1 + E1) + lam2_hat*(g2 + E2) )
            rhs[0] = 1.0; rhs[-1] = 0.0
            f_new = solve_tridiag(sub, diag, sup, rhs)
            f_new[0] = 1.0; f_new[-1] = 0.0
            if np.max(np.abs(f_new - f)) < inner_tol:
                f = f_new
                break
            f = f_new
        err = np.max(np.abs(f - f_old))
        if err < tol:
            break
    return f


In [2]:
def talbot_nodes_weights(T, N=32):
    """
    Classic Talbot nodes/weights for inverting F(T) = L^{-1}[Phi(s)](T).
    Returns s_k (complex) and w_k (complex) such that
       F(T) ≈ Re( sum_k w_k * Phi(s_k) )
    """
    k = np.arange(1, N+1, dtype=float)
    theta = (k - 0.5) * np.pi / N
    # contour and derivative
    z = theta / np.tan(theta) + 1j * theta             # dimensionless
    dz_dtheta = -theta / (np.sin(theta)**2) + 1/np.tan(theta) + 1j
    s = (N / T) * z
    ds_dtheta = (N / T) * dz_dtheta
    # trapezoidal weight factor
    dtheta = np.pi / N
    w = (np.exp(s * T) * ds_dtheta * dtheta) / (2j * np.pi)
    return s, w

def F_cdf_talbot(T, phi_over_q, N=32):
    """
    Talbot inversion for F(T) given function phi_over_q(s) = phi(s)/s.
    Returns scalar F(T).
    """
    s, w = talbot_nodes_weights(T, N=N)
    vals = phi_over_q(s)                  # complex array
    return np.real(np.sum(w * vals))

# convenience: vectorized over T and y-grid
def cdf_talbot_on_mesh(T_vals, y_vals, N=32):
    """
    Returns CDF array of shape (len(y_vals), len(T_vals)).
    Uses PDE to evaluate phi(s;y) at Talbot nodes s and forms F(T;y).
    """
    T_vals = np.asarray(T_vals, float)
    y_vals = np.asarray(y_vals, float)
    # map y to grid indices
    m_idx = np.minimum(np.maximum((y_vals / Dy).round().astype(int), 0), M)

    # cache solutions across s to avoid repeated solves if T repeats nodes (we don't reuse here per T)
    F = np.zeros((len(y_vals), len(T_vals)), float)

    for jT, T in enumerate(T_vals):
        s_nodes, w_nodes = talbot_nodes_weights(T, N=N)

        # solve PDE at each complex s, grab f(y;s); accumulate weighted sum
        accum = np.zeros(len(y_vals), dtype=np.complex128)
        for sk, wk in zip(s_nodes, w_nodes):
            # phi(s) = E[e^{-s tau}] = f(y; q=s)
            f_s = solve_LT_at_q(sk)         # complex array over y grid
            phi_over_q_vals = f_s[m_idx] / sk
            accum += wk * phi_over_q_vals

        F[:, jT] = np.real(accum)
        # numerical hygiene
        F[:, jT] = np.clip(F[:, jT], 0.0, 1.0)

    return F


In [21]:
# y and T meshes
y_vals = np.linspace(0.01, 0.5, 5)
T_vals = np.linspace(0.01, 1, 10)

F_mesh = cdf_talbot_on_mesh(T_vals, y_vals, N=32)   # N=32..64 typical

# example readouts
iy = np.argmin(abs(y_vals - 0.20))
iT = np.argmin(abs(T_vals - 0.50))
print("P(τ<=0.5 | y=0.2) =", F_mesh[iy, iT])

P(τ<=0.5 | y=0.2) = 1.0


In [16]:
# example readouts
y = 0.4
T = 0.34
iy = np.argmin(abs(y_vals - y))
iT = np.argmin(abs(T_vals - T))
print(f"P(τ<={T:.2f} | y={y:.2f}) =", F_mesh[iy, iT])

P(τ<=0.34 | y=0.40) = 0.16228449419900667


In [20]:
F_mesh[:, 1]

array([0.94423964, 0.00760386])